#### Loading Json File

In [3]:
import json

# Sample schema JSON
schema_json = '''
{
    "tables": [
        {
            "name": "customers",
            "columns": [
                {
                    "name": "customer_id",
                    "type": "string",
                    "required": true,
                    "validation": {
                        "pattern": "^CUS[0-9]{6}$"
                    }
                },
                {
                    "name": "purchase_amount",
                    "type": "decimal",
                    "required": true,
                    "validation": {
                        "min": 0,
                        "max": 1000000
                    }
                }
            ]
        }
    ]
}
'''

# Load the schema
schema = json.loads(schema_json)



1. Create a function that extracts column details from the schema file:

* Use lambda functions to transform complex column definitions
* Create a mapping of table names to their column specifications


In [4]:
import json

def extract_column_details(schema_file):

    with open(schema_file, 'r') as f:
        schema = json.load(f)

    table_columns = {}
    for table in schema['tables']:
        table_columns[table['name']] = [
            {
                "name": col['name'],
                "type": col['type'],
                "required": col.get('required', False),
                "validation": col.get('validation', {})
            } for col in table['columns']
        ]
    return table_columns


2. Create a validation function that:
* Filters required columns using filter() and lambda
* Validates data types and constraints
* Returns validation errors in a structured format

In [5]:
def filter_column(columns):
    return list(filter(lambda x: x, columns))

In [6]:
def validate_data(data, table_schema):
    errors = []

    
    for row in data:
        for col in table_schema:
            col_name = col['name']
            col_type = col['type']
            col_required = col['required']

            
            if col_required and col_name not in row:
                errors.append({
                    "severity": "high",
                    "table": col.get('table_name', 'unknown'),
                    "column": col_name,
                    "error": "Missing required column"
                })
                continue

            
            if col_name in row and not isinstance(row[col_name], col_type):
                errors.append({
                    "severity": "medium",
                    "table": col.get('table_name', 'unknown'),
                    "column": col_name,
                    "error": "Incorrect data type"
                })

            
            if 'validation' in col:
                for rule, value in col['validation'].items():
                    if rule == 'min_length' and len(row[col_name]) < value:
                        errors.append({
                            "severity": "low",
                            "table": col.get('table_name', 'unknown'),
                            "column": col_name,
                            "error": f"Length is less than {value}"
                        })

    return errors


3. Create a function that sorts validation errors by:
* Error severity
* Table name
* Column name

In [7]:
def sort_validation_errors(errors):
    sorted_errors = sorted(errors, key=lambda x: (x['severity'], x['table'], x['column']))
    return sorted_errors
